# Covid-19 data analysis for the Netherland
## uses covid data to learn datascience in pyhton woth Jupyterlab

*Goals*
1. learning datawcience
2. learning python
3. learning Jupyterlab
4. maybe get some insight in covid

In [163]:
#from IPython.display import Markdown as md
import numpy as np
import pandas as pd
import tabula
from datetime import datetime

## Collecting data
The data I use is the sources used formhet Coronadashbord of the Dutch government. https://coronadashboard.rijksoverheid.nl/
  
This contains:
*medical indicators*

In [97]:
print("- number of positive tests")
# number of positive tests (https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427)
pt = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.json')
print(pt.head())
print(pt)

- number of positive tests
        Date_of_report Date_of_publication Municipality_code  \
0  2020-10-14 10:00:00          2020-02-27            GM0003   
1  2020-10-14 10:00:00          2020-02-27            GM0010   
2  2020-10-14 10:00:00          2020-02-27            GM0014   
3  2020-10-14 10:00:00          2020-02-27            GM0024   
4  2020-10-14 10:00:00          2020-02-27            GM0034   

  Municipality_name   Province Security_region_code Security_region_name  \
0        Appingedam  Groningen                 VR01            Groningen   
1          Delfzijl  Groningen                 VR01            Groningen   
2         Groningen  Groningen                 VR01            Groningen   
3         Loppersum  Groningen                 VR01            Groningen   
4            Almere  Flevoland                 VR25            Flevoland   

  Municipal_health_service                        ROAZ_region  Total_reported  \
0            GGD Groningen  Acute Zorgnetwerk Noor

In [44]:
print("- Percentage positive tested of all tests done")
# https://www.rivm.nl/archief-weekrapportages-covid-19-in-nederland
pt_perc = tabula.read_pdf('https://www.rivm.nl/sites/default/files/2020-10/COVID-19_WebSite_rapport_wekelijks_20201013_1159_0.pdf', pages=29, guess=True, stream=True)
print(pt_perc)

## To DO:
## 1) extract data from latest pdf
## 2) scrape all pdfs for data to compare data thought time

[   Weeknummer Totaal aantal testen  Aantal positief  Percentage positief
0         NaN          met uitslag              NaN                  NaN
1          23                48812            986.0                  2.0
2          24                57013            841.0                  1.5
3          25                61604            567.0                  0.9
4          26                61222            426.0                  0.7
5          27                67248            375.0                  0.6
6          28                74919            463.0                  0.6
7          29                88508            925.0                  1.0
8          30               111416           1195.0                  1.1
9          31               101471           2411.0                  2.4
10         32                98621           3681.0                  3.7
11         33               102921           3719.0                  3.6
12         34               135723           3398.

In [45]:
print("- number of infectious people")
# https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/097155aa-75eb-4caa-8ed3-4c6edb80467e
infected = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_prevalentie.json')
print(infected.head())

        Date  prev_low  prev_avg  prev_up population
0 2020-02-17       575    1036.0     1553       hosp
1 2020-02-18       863    1359.0     1955       hosp
2 2020-02-19      1150    1740.0     2358       hosp
3 2020-02-20      1553    2211.0     2875       hosp
4 2020-02-21      2070    2793.0     3565       hosp


In [46]:
print("- R (reproductionnumber)")
# https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
R = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_reproductiegetal.json')
print(R.head())

        Date  Rt_low  Rt_avg  Rt_up population
0 2020-02-17    0.50    1.93   4.00       hosp
1 2020-02-18    0.00    1.74   4.00       hosp
2 2020-02-19    0.50    1.59   2.83       hosp
3 2020-02-20    0.83    1.74   2.83       hosp
4 2020-02-21    0.67    2.03   3.67       hosp


In [47]:
print("- hospital admissions per day")
# NICE https://www.databronnencovid19.nl/Bron?naam=Nationale-Intensive-Care-Evaluatie
hosp_nice = pd.read_json('https://stichting-nice.nl/covid-19/public/intake-count/')
print(hosp_nice.head())
# LCPS https://lcps.nu/datafeed/
hosp_lcps = pd.read_csv('https://lcps.nu/wp-content/uploads/covid-19.csv')
print(hosp_lcps.head())
# Dashbaord changed source data https://www.nu.nl/coronavirus/6083846/ministerie-meldde-maandenlang-veel-te-weinig-opnames-coronapatienten.html
# other sources: https://www.stichting-nice.nl/covid-19-op-de-zkh.jsp

        date  value
0 2020-02-27      8
1 2020-02-28      9
2 2020-02-29     10
3 2020-03-01     12
4 2020-03-02     11
        Datum  IC_Bedden_COVID  IC_Bedden_Non_COVID  Kliniek_Bedden
0  01-06-2020              157                  488             542
1  02-06-2020              153                  476             516
2  03-06-2020              116                  514             496
3  04-06-2020              113                  567             481
4  05-06-2020               97                  543             469


In [48]:
print("- icu admissions per day")
# url
ic1 = pd.read_json('https://stichting-nice.nl/covid-19/public/new-intake/confirmed/')
print(ic1.head())
ic2 = pd.read_json('https://stichting-nice.nl/covid-19/public/intake-count/')
print(ic2.head())

## To DO: conpare ic1 and ic2
#other sources: 

        date  value
0 2020-02-27      1
1 2020-02-28      0
2 2020-02-29      0
3 2020-03-01      2
4 2020-03-02      0
        date  value
0 2020-02-27      8
1 2020-02-28      9
2 2020-02-29     10
3 2020-03-01     12
4 2020-03-02     11


In [49]:
## sources NICE for their website:
# /covid-19/public/zkh/global
# /covid-19/public/zkh/new-intake/
# /covid-19/public/zkh/intake-count/
# /covid-19/public/zkh/intake-cumulative/
# /covid-19/public/zkh/died-and-survivors-cumulative/
# /covid-19/public/zkh/age-distribution-died-and-survivors/
# /covid-19/public/zkh/age-distribution-status/
# /covid-19/public/zkh/behandelduur-distribution/
# /covid-19/public/zkh/behandelduur-distribution/
## TO DO: check which used and which to use extra.
## sources LCPS?
# https://lcps.nu/wp-content/uploads/
## Maybe scrape website to find more public data: https://scrapy.org/ OR https://www.crummy.com/software/BeautifulSoup/bs4/doc/

*early signs*

In [50]:
print("- patient reporting covid symptoms at family doctor")
# https://www.nivel.nl/nl/nivel-zorgregistraties-eerste-lijn/nivel-zorgregistraties-eerste-lijn


- patient reporting covid symptoms at family doctor

In [51]:
print("- sewagewate")
# https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/a2960b68-9d3f-4dc3-9485-600570cd52b9
sw = pd.read_json('https://data.rivm.nl/covid-19/COVID-19_rioolwaterdata.json')
print(sw.head())

  Date_measurement  RWZI_AWZI_code RWZI_AWZI_name  X_coordinate  Y_coordinate  \
0       2020-03-30           32002        Tilburg        132554        401565   
1       2020-03-31            3010          Assen        234705        558203   
2       2020-03-31            1033    Garmerwolde        240219        585040   
3       2020-03-31            2032      Harlingen        159027        576146   
4       2020-03-31            2011     Leeuwarden        184401        578940   

  Postal_code Security_region_code     Security_region_name  \
0      5048TD                 VR20  Midden- en West-Brabant   
1      9403AK                 VR03                  Drenthe   
2      9798TC                 VR01                Groningen   
3      8861KT                 VR02                Friesland   
4      8937AN                 VR02                Friesland   

  Percentage_in_security_region  RNA_per_ml  Representative_measurement  
0                             1        1837                 

*homes for elderly*

In [52]:
print("- positive testst")
# from pdf. https://www.rivm.nl/documenten/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland

- positive testst

In [53]:
print("- infected locations")
# from pdf. https://www.rivm.nl/documenten/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland

- infected locations

In [54]:
print("- number deseased")
# from pdf. https://www.rivm.nl/documenten/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland

- number deseased

**metadata**

In [55]:
print("- municipalities in Netherlands")
## https://www.cbs.nl/nl-nl/onze-diensten/methoden/classificaties/overig/gemeentelijke-indelingen-per-jaar/indeling%20per%20jaar/gemeentelijke-indeling-op-1-januari-2020
municipality = pd.read_excel('https://www.cbs.nl/-/media/_excel/2020/03/gemeenten-alfabetisch-2020.xlsx')
print(municipality.head())
# aantallen per gemeente: https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427


   Gemeentecode GemeentecodeGM   Gemeentenaam  Provinciecode ProvinciecodePV  \
0          1680         GM1680    Aa en Hunze             22            PV22   
1           358         GM0358       Aalsmeer             27            PV27   
2           197         GM0197         Aalten             25            PV25   
3            59         GM0059  Achtkarspelen             21            PV21   
4           482         GM0482   Alblasserdam             28            PV28   

   Provincienaam  
0        Drenthe  
1  Noord-Holland  
2     Gelderland  
3      Friesland  
4   Zuid-Holland  


### missing data, i would like to calculate of collect
- covid numbers relitive to all data
- deseased for whole country (possible source: https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/1c0fcd57-1102-4620-9cfa-441e93ea5604)

### more data at
- Karakteristieken elke geteste persoon: https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/2c4357c8-76e4-4662-9574-1deb8a73f724?tab=relations
- https://data.rivm.nl
- https://opendata.cbs.nl

In [56]:
#check data collection
from pivottablejs import pivot_ui
from IPython.display import HTML

df_to_check = ic1

pivot_ui(df_to_check, outfile_path='pivottablejs.html')
HTML('pivottablejs.html')

In [289]:
# Load Bokeh
#charts with Bokeh https://docs.bokeh.org/
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import viridis
output_notebook()

Loading BokehJS ...

In [226]:
# SINGLE LINE TEST
# create a new plot (with a title) using figure
p = figure(plot_width=800, plot_height=400, x_axis_type="datetime", title="IC1")
# add a line renderer 
p.line(ic1['date'], ic1['value'])
# Set autohide to true to only show the toolbar when mouse is over plot
p.toolbar.autohide = True

In [227]:
## MULTIPLE LINE TEST
pp = figure(title="IC1 & IC2", plot_width=800, plot_height=400, x_axis_type="datetime")

pp.multi_line([ic1.date, ic2.date],
              [ic1.value, ic2.value],
              color=["firebrick", "navy"])

GlyphRenderer(id='33190', ...)

In [228]:
show(p)
show(pp)

In [229]:
# TESTING BOKEH WITH OTHER DATA
url = 'https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.json'
rivm = pd.read_json(url)

In [230]:
rivm['Date_of_report'] = pd.to_datetime(rivm['Date_of_report']).dt.date
#rivm['Date_of_report'] = rivm['Date_of_report'].dt.date
rivm

,Date_of_report,Municipality_code,Municipality_name,Province,Total_reported,Hospital_admission,Deceased
0,2020-03-13,GM0003,Appingedam,Groningen,0,0,0
1,2020-03-13,GM0010,Delfzijl,Groningen,0,0,0
2,2020-03-13,GM0014,Groningen,Groningen,3,0,0
3,2020-03-13,GM0024,Loppersum,Groningen,0,0,0
4,2020-03-13,GM0034,Almere,Flevoland,1,1,0
...,...,...,...,...,...,...,...
79267,2020-10-14,None,None,Noord-Holland,227,16,2
79268,2020-10-14,None,None,Zuid-Holland,571,18,7
79269,2020-10-14,None,None,Zeeland,10,3,1
79270,2020-10-14,None,None,Noord-Brabant,333,16,3


In [295]:
rivm_agg = rivm.groupby(by=['Date_of_report'],as_index=False).sum()

q = figure(plot_width=800, plot_height=400, title="My Line Plot", x_axis_type="datetime")

q.multi_line(
    xs=[
        rivm_agg['Date_of_report'],
        rivm_agg['Date_of_report'],
        rivm_agg['Date_of_report']
    ],
    ys=[
        rivm_agg['Deceased'], 
        rivm_agg['Hospital_admission'],
        rivm_agg['Total_reported']
    ],
    color=viridis(len(rivm_agg.columns)-1)
)

show(q)